In [1]:
from aurelio.plotter import plot
import os

## Curvas de Aprendizado utilizando K-fold com ELMo e Word Embeddings de 600 dimensões

In [2]:
metrics_elmo_path = "{}/metrics/elmo".format(os.getcwd())
plot(metrics_elmo_path)
# metrics_elmo_path

/home/helio/miniconda3/envs/allen/lib/python3.7/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



## Curvas de Aprendizado utilizando K-fold sem ELMo e Word Embeddings de 600 dimensões

In [3]:
metrics_no_elmo_path = "{}/metrics/no_elmo".format(os.getcwd())
plot(metrics_no_elmo_path)